# Chapter 8: Dimensionality Reduction

In [1]:
import numpy as np

*Curse of dimensionality* - Each training instance has thousands or millions of features, which makes training extremely slow and much harder to find a good solution.

> Note: Reducing dimensionality does cause some information loss (similar to image compression). So if training is too slow, you should first try to train your system with the original data before considering using dimensionality reduction.

Dimensionality reduction is also useful for data visualization (DataViz). If data is reduced to 2-3 dimensions, you can plot it for any additional patterns or insight. Also great for presentations to non-data scientists.

## 8.1 The Curse of Dimensionality

Average distance in N-Dimension Comparison:

- 2D (unit square): 0.52
- 3D (unit cube): 0.66
- 1,000,000-D (unit hypercube): 408.25

Conclusions:

- There's just plenty of space in high dimensions.
- High-dimension datasets are at risk of being very sparse (ie. most training instances would be far away from each other).
- The more dimensions the training set has, the greater risk of overfitting.

> Recall: To solve high bias, add more and/or complex features.

## 8.2 Main Approaches for Dimensionality Reduction

### 8.2.1 Projection

Not all training instances are spread uniformly across all dimensions; some are constant and some are highly correlated with each other.  
=> All training instances lie within or close to a much lower-dimensional subspace.

Suppose all training instance of 3-D dataset lie close to a plane. If we project each instance onto the plane, we have successfully reduced from 3-D to 2-D.

But sometimes it's not the best idea if the dataset "twists or turns" because in doing so, the 2-D projections may overlap with each other instead of "unrolling" each section.

> Note: See Figures 8-4 and 8-5 in the book for pictorial reference.

### 8.2.2 Manifold Learning

A d-dimensional manifold is a part of an n-dimensional space (where d < n) that locally resembles a d-dimensional hyperplane.  
In Swiss roll example, d=2 & n=3, so it resembles a 2-D hyperplane rolled up in 3-D.

Key assumptions for **Manifold Learning**:

1. *Manifold assumption (manifold hypothesis)* - Most real-world high-dimensional datasets lie close to a much lower-dimensional manifold. Constraints tend to squeeze the dataset into a lower-dimensional manifold.

2. The task at hand (eg. classification, regression) will be simpler if expressed in the lower-dimensional space of the manifold.

> Note: Assumption #2 doesn't always hold. Sometimes, going from 3-D to 2-D results in complex -> simple decision boundary. Other times it results in simple -> complex. See Figure 8-6 in book for pictorial reference.

## 8.3 PCA

*Principal Component Analysis (PCA)* - The most popular dimensionality reduction algorithm, that identifies the hyperplane closest to the data and then projects the data onto it.

### 8.3.1 Preserving the Variance

When projecting down onto a lower-dimensional subspace, it is best to select the axis that preserves the maximum amount of variance, as it will most likely lose the least information.

> Recall: Variance is the spread of the dataset. So maximizing its variance preserves the looks or characteristics of the original dataset.

It can  also be thought of as selecting an axis that minimizes the mean squared distance between the original dataset and its projection onto that axis.

### 8.3.2 Principal Components

The $i^{th}$ axis is called the *$i^{th}$ principal component (PC)*.  
To find the principal components, use Singular Value Decomposition (SVD) to decompose the training set matrix **X** into $ \mathbf{U} \times \mathbf{\sum} \times \mathbf{V^T} $ , where $ \mathbf{V} $ contains the unit vectors that define the principal components.

In [2]:
# COPIED FROM ACCOMPANYING NOTEBOOK

np.random.seed(4)
m = 60
w1, w2 = 0.1, 0.3
noise = 0.1

angles = np.random.rand(m) * 3 * np.pi / 2 - 0.5
X = np.empty((m, 3))
X[:, 0] = np.cos(angles) + np.sin(angles)/2 + noise * np.random.randn(m) / 2
X[:, 1] = np.sin(angles) * 0.7 + noise * np.random.randn(m) / 2
X[:, 2] = X[:, 0] * w1 + X[:, 1] * w2 + noise * np.random.randn(m)

In [3]:
X_centered = X - X.mean(axis=0) # Center dataset around origin
U, s, Vt = np.linalg.svd(X_centered)
c1 = Vt.T[:, 0] # First PC
c2 = Vt.T[:, 1] # Second PC

> Note: PCA assumes the dataset is centered around the origin. Scikit-Learn's `PCA` does this for you. Make sure to center the data beforehand, otherwise.

### 8.3.3 Projecting Down to d Dimensions

You can reduce the dimensionality of the dataset down to *d* dimensions by projecting it onto the hyperplane defined by the first *d* principal components.

To do this, take the training set and matrix multiply it with the matrix containing the first *d* columns of $ \mathbf{V}$. Other words,

*Equation 8-2. Projecting the training set down to d dimensions*

$$ \mathbf{X}_{d-proj} = \mathbf{X W}_d $$

In [4]:
W2 = Vt.T[:, 2]
X2D = X_centered @ W2 # (@ or matmul) is preferred over .dot for 
                      # matrix multiplication

### 8.3.4 Using Scikit-Learn

Scikit-Learn's `PCA` class uses SVD decomposition to implement PCA as well as centering the data.

In [5]:
from sklearn.decomposition import PCA

In [6]:
pca = PCA(n_components=2)
X2D = pca.fit_transform(X)

`components_` attribute holds the transpose of $\mathbf{W}_d$.

In [7]:
pca.components_

array([[-0.93636116, -0.29854881, -0.18465208],
       [ 0.34027485, -0.90119108, -0.2684542 ]])

In [8]:
pca.components_.T[:, 0] # First PC

array([-0.93636116, -0.29854881, -0.18465208])

### 8.3.5 Explained Variance Ratio

The *explained variance ratio* indicates the proportion of the dataset's variance lies along each principal component, via `explained_variance_ratio` variable.

In [9]:
pca.explained_variance_ratio_

array([0.84248607, 0.14631839])

84.2% lies along the first PC.  
14.6% lies along the second PC.  
<1.2% lies along the third PC => probably carries little information.

### 8.3.6 Choosing the Right Number of Dimensions

Alternatively, instead of choosing a particular number of dimensions to reduce to, choose the number of dimensions that add up to a sufficiently large proportion of the variance (eg. 95%).

In [10]:
# FROM TEXTBOOK NOTEBOOK

from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1)
mnist.target = mnist.target.astype(np.uint8)

In [11]:
# FROM TEXTBOOK NOTEBOOK

from sklearn.model_selection import train_test_split

X = mnist["data"]
y = mnist["target"]

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [14]:
pca = PCA()
pca.fit(X_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.95) + 1
d

154

Now we can set `n_components=d` and run PCA again.

Another (better) option is to set `n_components=` float between 0.0 and 1.0, indicating the ratio of variance to preserve.

In [16]:
pca = PCA(n_components=0.95)
X_reduced = pca.fit_transform(X_train)

print('Before', X_train.shape)
print('After being reduced', X_reduced.shape)

Before (52500, 784)
After being reduced (52500, 154)


Another option is to plot the explained variance as a function of number of dimensions (ie. plot `cumsum`). There will usually be an elbow in the curve, and so pick the number of dimensions at which the curve stops growing.

> Note: See Figure 8-8 in book for pictorial reference.

### 8.3.7 PCA for Compression

As we saw, PCA reduced MNIST dataset from 784 -> 154 features (now less than 20% of original size).

It is possible to decompress the dataset back to 784 dimensions but it won't be a perfect reconstruction due to information loss from the compression operation.

*Reconstuction error* - The mean squared distance between the original data and the reconstructed data (compressed -> decompressed).

In [17]:
pca = PCA(n_components=154)
X_reduced = pca.fit_transform(X_train)
X_recovered = pca.inverse_transform(X_reduced)

> Note: See Figure 8-9 in book for comparison between original images and reconstructed images.

*Equation 8-3. PCA inverse transformation, back to original d-dimensions*

$$ \mathbf{X}_{recovered} = \mathbf{X}_{d-proj} \mathbf{W}_d^T $$

### 8.3.8 Randomized PCA

If you set `svd_solver="randomized"`, Scikit-Learn uses a stochastic algorithm called *Randomized PCA* that quickly finds an approximation of the first *d* principal components.

Computational complexity:  
$ O(m \times d^2) + O(d^3) $ instead of $ O(m \times n^2) + O(n^3) $, where $d<n$

In [18]:
rnd_pca = PCA(n_components=154, svd_solver="randomized")
X_reduced = rnd_pca.fit_transform(X_train)

By default, `svd_solver="auto"`, meaning Scikit-Learn uses randomized PCA if m or n > 500 and d < 80% of m or n. It uses full SVD otherwise.

Set `svd_solver="full"` to force Scikit-Learn to use full SVD.

### 8.3.9 Incremental PCA

Our current PCA approach requires whole training set to fit in memory. *Incremental PCA (IPCA)* algorithms allow you to split the training set into mini-batches and feed an IPCA algorithm one mini-batch at a time.

- NumPy's `array_split()` splits the MNIST dataset into 100 mini-batches.  
- Scikit-Learn's `IncrementalPCA` reduces the dimensionality to 154.  
- `partial_fit()` can be called for each mini-batch instead of `fit()` for whole training set.

In [19]:
from sklearn.decomposition import IncrementalPCA

In [20]:
n_batches = 100
inc_pca = IncrementalPCA(n_components=154)
for X_batch in np.array_split(X_train, n_batches):
    inc_pca.partial_fit(X_batch)
    
X_reduced = inc_pca.transform(X_train)

Alternatively you can use NumPy's `memmap` class which allow you to manipulate a large array stored in a binary file on disk as if it were entirely in memory (ie. the class loads only the data it needs in memory when it needs it).

Using this approach, you can call the usual `fit()` method.

In [22]:
# FROM TEXTBOOK NOTEBOOK

filename = "my_mnist.data"
m, n = X_train.shape

X_mm = np.memmap(filename, dtype='float32', mode='write', shape=(m, n))
X_mm[:] = X_train

In [23]:
X_mm = np.memmap(filename, dtype="float32", mode="readonly", shape=(m, n))

batch_size = m // n_batches
inc_pca = IncrementalPCA(n_components=154, batch_size=batch_size)
inc_pca.fit(X_mm)

IncrementalPCA(batch_size=525, n_components=154)

## 8.4 Kernel PCA

> Recall: Support Vector Machine's kernel trick in which it implicitly maps instances into a very high dimensional space, enabling nonlinear classification and regression.

*Kernel PCA (kPCA)* - Applying the same trick as with SVM's kernel trick to perform complex nonlinear projections for dimensionality reduction.

In [13]:
# FROM TEXTBOOK NOTEBOOK

from sklearn.datasets import make_swiss_roll
X, t = make_swiss_roll(n_samples=1000, noise=0.2, random_state=42)
y = t > 6.9

In [3]:
from sklearn.decomposition import KernelPCA

In [17]:
rbf_pca = KernelPCA(n_components=2, kernel="rbf", gamma=0.04)
X_reduced = rbf_pca.fit_transform(X)

### 8.4.1 Selecting a Kernel and Tuning Hyperparameters

kPCA is an unsupervised learning algorithm, so there is no obvious performance measure to help select the best kernel and hyperparameter values.

But since dimensionality reduction is often used as preparation step before supervised learning, you can still use grid search to select the best kernel and hyperparameters that leads to best performance on the supervised task.

In [5]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [9]:
clf = Pipeline([
    ("kpca", KernelPCA(n_components=2)),
    ("log_reg", LogisticRegression())
])
param_grid = [{
    "kpca__gamma": np.linspace(0.03, 0.05, 10), # NOTE: double underscore __
    "kpca__kernel": ["rbf", "sigmoid"]
}]

grid_search = GridSearchCV(clf, param_grid, cv=3)
grid_search.fit(X, y)
print(grid_search.best_params_)

{'kpca__gamma': 0.043333333333333335, 'kpca__kernel': 'rbf'}


Another approach is to select the kernel and hyperparameters that yield the lowest reconstruction error.

This is much harder than with linear PCA. The kernel trick transformation is equivalent to using the *feature map* $ \varphi$ to map the training set to an $ \infty$-dimensional feature space, then projecting the transformed training set down to 2D using linear PCA.

> Note: If we reverse linear PCA step, the reconstructed instance is in the $ \infty$-dimensional feature space, not in the original space, and therefore cannot compute the reconstruction error.

But it is possible to find a point (called *"reconstruction pre-image"*) in the original space that would map close to the reconstructed point. Then measure its squared distance from the original instance, and select the best kernels/hyperparameters that minimize this.

We can do this by training a supervised regression model, with the projected instances as the training set and the original instances as the targets and using `fit_inverse_transform=True`.

In [14]:
rbf_pca = KernelPCA(n_components=2, kernel="rbf", gamma=0.0433,
                    fit_inverse_transform=True)
X_reduced = rbf_pca.fit_transform(X)
X_preimage = rbf_pca.inverse_transform(X_reduced)

In [15]:
from sklearn.metrics import mean_squared_error

In [17]:
mean_squared_error(X, X_preimage)

7.642296115403016e-27

## 8.5 LLE

## 8.6 Other Dimensionality Reduction Techniques